In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from cpc import CPClayer, Classic
import random
from tqdm import  tqdm

In [2]:
device = 'cuda'

In [3]:
train_data = datasets.CIFAR10(root="./cpc_data",train=True, download= True, transform= ToTensor())
test_data = datasets.CIFAR10(root = "./cpc_data", train= False, download=True, transform=ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# train_data = datasets.MNIST(root = "./mnist", train = True, download = True, transform=ToTensor())
# test_data = datasets.MNIST(root = "./mnist", train = False, download = True, transform = ToTensor())

In [5]:
cl = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: []
}

for i in range(len(train_data)):
    cl[train_data[i][1]].append(train_data[i][0])

In [6]:
test_loader = DataLoader(test_data, 64, shuffle=True)
train_loader = DataLoader(train_data, 64, shuffle=True)

In [7]:
# from resnet import res_for_cifar
# resnet = res_for_cifar().to(device)
# optimizer = torch.optim.SGD(resnet.parameters(),lr=0.01)
# loss = torch.nn.CrossEntropyLoss()

# for i in tqdm(range(100)):
#     for data, label in train_loader:
#         data = data.to(device)
#         label = label.to(device)
#         out = resnet(data)

#         l = loss(out,label)
#         optimizer.zero_grad()
#         l.backward()
#         optimizer.step()


# ac = 0
# length = 0
# for data, label in tqdm(test_loader):
#     data = data.to(device)
#     label = label.to(device)
#     out = resnet(data)

#     out = torch.argmax(out, dim = 1)

#     ac += len(torch.where(out == label)[0])
#     length += len(data)

# print(ac/length)



100%|██████████| 157/157 [00:00<00:00, 298.83it/s]

0.7127


In [8]:
def seed_all(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_all(69)

In [9]:
def sample_train_pred(batch_size, train_len, pred_len, pos_num):
    output = torch.zeros([batch_size,train_len+pred_len,3,32,32])
    label = torch.zeros([batch_size,1])
    pos = pos_num
    for i in range(batch_size):
        start = random.sample(range(0,10),1)[0]
        sequence = (torch.arange(start, start+train_len+pred_len))%10
        if pos > 0:
            temp = torch.zeros([train_len+pred_len,3,32,32])
            for j in range(len(sequence)):
                cls_len = len(cl[sequence[j].item()])
                sampling = random.sample(range(0,cls_len),1)[0]
                temp[j] = cl[sequence[j].item()][sampling]
            output[i] = temp
            label[i] = 1
        else:
            nums = torch.tensor(range(0,10))
            temp = torch.zeros([train_len+pred_len,3,32,32])
            for j in range(train_len):
                cls_len = len(cl[sequence[j].item()])
                sampling = random.sample(range(0,cls_len),1)[0]
                temp[j] = cl[sequence[j].item()][sampling]

            for j in range(train_len,train_len+pred_len):
                new = random.choice(nums[nums != sequence[j]])
                cl_len = len(cl[new.item()])
                sampling = random.sample(range(0,cl_len),1)[0]
                temp[j] = cl[new.item()][sampling]
            output[i] = temp
            label[i] = 0
        pos -= 1

    indicies = torch.randperm(output.size(0))

    shuffled_out = output[indicies]
    shuffled_label = label[indicies]


    return shuffled_out[:,:train_len,:,:,:], shuffled_out[:,train_len:,:,:,:], shuffled_label



In [10]:
from torch.optim.lr_scheduler import StepLR
epoch = 50
train_len = 2
pred_len = 2
batch_size = 60
pos_num = 1

cpc = CPClayer(3,256,128, pred_len)

optimizer = cpc.regist_param()
shceduler = StepLR(optimizer, 10,0.1)
for i in tqdm(range(epoch)):
    train_batch, pred_batch, label = sample_train_pred(batch_size, train_len, pred_len, pos_num)
    cpc.train(train_batch, pred_batch, label)
    shceduler.step()


clasific = Classic(128)

for params in cpc.parameters():
    params.requires_grad = False

cls_epoch = 10

for j in tqdm(range(cls_epoch)):
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        feature = cpc(data,True)
        feature = feature.squeeze()
        clasific.train(feature, label)
ac_len = 0
length = 0
for data, label in tqdm(test_loader):
    data = data.to(device)
    label = label.to(device)
    with torch.no_grad():
        feature = cpc(data,True)
        feature = feature.squeeze()
        pred = clasific(feature)
        result = torch.argmax(pred, dim = 1)
        ac_len += len(torch.where(result == label)[0])
        length += len(data)

ac = ac_len/length

print(f"The test accuracy is {ac}")





  2%|▏         | 1/50 [00:00<00:22,  2.21it/s]

tensor(4.0853, device='cuda:0', grad_fn=<NegBackward0>)


  4%|▍         | 2/50 [00:00<00:15,  3.11it/s]

tensor(4.0871, device='cuda:0', grad_fn=<NegBackward0>)


  6%|▌         | 3/50 [00:00<00:13,  3.53it/s]

...

tensor(4.0874, device='cuda:0', grad_fn=<NegBackward0>)


100%|██████████| 50/50 [00:11<00:00,  4.25it/s]


tensor(4.1026, device='cuda:0', grad_fn=<NegBackward0>)


100%|██████████| 157/157 [00:00<00:00, 286.73it/s]

The test accuracy is 0.3323
